In [1]:
import pandas as pd
import statsmodels.formula.api as sm
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [14]:
duckdf = duckdb.sql("FROM read_parquet('/home/benjamin.cretois/data/s3mount/bencretois-ns8129k-proj-tabmon/parquet_results/*/*/*.parquet', hive_partitioning=true)")

duckdf_sub = duckdb.sql("""
SELECT * FROM duckdf 
USING SAMPLE 100000 ROWS
""")

┌──────────────────────────────┬────────────┬──────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
duckdb.sql("FROM duckdf_sub LIMIT 5").show()

In [ ]:
filtered_data = duckdb.sql("""
SELECT 
    *,
    DATE_TRUNC('second', 
        STRPTIME(
            REPLACE(
                REPLACE(
                    REPLACE(
                        REGEXP_EXTRACT(filename, '(.*)\\.mp3$', 1),
                        'T', ' '
                    ),
                    '_', ':'
                ),
                'Z', ''
            ),
            '%Y-%m-%d %H:%M:%S.%f'
        )
    ) AS time_recorded,
    energy['ACI'] AS ACI,
    energy['ADI'] AS ADI,
    energy['BI'] AS BI,
    energy['EVNspCount'] AS EVNspCount,
    energy['EVNspMean'] AS EVNspMean,
    energy['Hf'] AS Hf,
    energy['NDSI'] AS NDSI
FROM duckdf_sub
""")

duckdb.sql("SELECT filename, time_recorded FROM filtered_data LIMIT 5").show()

┌──────────────────────────────┬─────────────────────┐
│           filename           │    time_recorded    │
│           varchar            │      timestamp      │
├──────────────────────────────┼─────────────────────┤
│ 2025-02-24T20_21_59.767Z.mp3 │ 2025-02-24 20:21:59 │
│ 2025-02-27T22_31_23.896Z.mp3 │ 2025-02-27 22:31:23 │
│ 2025-03-10T19_15_11.979Z.mp3 │ 2025-03-10 19:15:11 │
│ 2025-03-09T07_01_34.347Z.mp3 │ 2025-03-09 07:01:34 │
│ 2025-03-04T11_26_07.889Z.mp3 │ 2025-03-04 11:26:07 │
└──────────────────────────────┴─────────────────────┘



In [21]:
weekly_aggregated = duckdb.sql("""
    SELECT 
        device_id,
        country,
        lat,
        CAST(DATE_TRUNC('week', time_recorded) AS VARCHAR) AS week_str,
        AVG(uncertainty) AS uncertainty,
        AVG(ACI) AS ACI,
        AVG(ADI) AS ADI,
        AVG(BI) AS BI,
        AVG(Hf) AS Hf,
        AVG(NDSI) AS NDSI,
        AVG(EVNspCount) AS EVNspCount
    FROM filtered_data
    GROUP BY device_id, country, lat, DATE_TRUNC('week', time_recorded)
    ORDER BY lat DESC, device_id
""")

agg_df = weekly_aggregated.df()
agg_df['week'] = pd.to_datetime(agg_df['week_str']).dt.date

In [24]:
agg_df['device_label'] = agg_df['device_id'] + ' (' + agg_df['country'] + ')'
device_labels = agg_df.sort_values('lat', ascending=False)[['device_label', 'device_id']].drop_duplicates()

In [25]:
import plotly.graph_objects as go

fig = go.Figure()

# Add a heatmap trace for each acoustic index
acoustic_indices = ['uncertainty', 'ACI', 'ADI', 'BI', 'Hf', 'NDSI', 'EVNspCount']

for i, index in enumerate(acoustic_indices):
    # Create pivot table but keep the ordering by latitude
    pivot_data = agg_df.pivot_table(
        values=index,
        index='device_id',
        columns='week',
        aggfunc='mean'
    )
    
    # Reorder the pivot table rows according to latitude
    pivot_data = pivot_data.reindex(device_labels['device_id'])
    
    # Set visibility
    visible = True if i == 0 else False
    
    # Add trace with ordered labels
    fig.add_trace(
        go.Heatmap(
            z=pivot_data.values,
            x=pivot_data.columns,
            y=device_labels['device_label'],  # Use the labels with country
            colorscale='Viridis',
            visible=visible,
            colorbar=dict(title=index),
            hovertemplate="Device: %{y}<br>Week: %{x}<br>Value: %{z}<extra></extra>"
        )
    )

# Create buttons for dropdown menu
buttons = []
for i, index in enumerate(acoustic_indices):
    button = dict(
        method="update",
        label=index,
        args=[
            {"visible": [j == i for j in range(len(acoustic_indices))]},
            {"title": f"{index} by Device Over Time"}
        ]
    )
    buttons.append(button)

# Add dropdown menu
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.58,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )
    ],
    title="Acoustic Index by Device Over Time",
    height=800,
    width=1200,
    xaxis_title="Week",
    yaxis_title="Device ID by Latitude (Country)",
    margin=dict(l=200, r=20, t=100, b=70),  # Increased left margin for longer labels
)

# Add a label for the dropdown
fig.add_annotation(
    text="Select Acoustic Index:",
    x=0.5,
    y=1.12,
    xref="paper",
    yref="paper",
    showarrow=False
)

# Show figure
fig.show()

In [29]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def create_uncertainty_vs_indices_plotly(df, indices):
    # Get list of unique countries
    countries = df['country'].unique().tolist()
    
    # Create subplots - one for each acoustic index
    n_cols = 3
    n_rows = (len(indices) + n_cols - 1) // n_cols
    
    fig = make_subplots(rows=n_rows, cols=n_cols, 
                        subplot_titles=[f'Uncertainty vs {index}' for index in indices])
    
    # Add traces for each country (initially all invisible except first country)
    for c_idx, country in enumerate(countries):
        country_df = df[df['country'] == country]
        
        for i, index in enumerate(indices):
            row = i // n_cols + 1
            col = i % n_cols + 1
            
            # Only calculate correlation if we have enough data points
            if len(country_df) > 2:
                corr = country_df[['uncertainty', index]].corr().iloc[0, 1]
                corr_text = f'r = {corr:.3f}'
            else:
                corr_text = 'insufficient data'
            
            # Add scatter plot
            visible = True if c_idx == 0 else False
            
            fig.add_trace(
                go.Scatter(
                    x=country_df[index],
                    y=country_df['uncertainty'],
                    mode='markers',
                    marker=dict(
                        size=8,
                        opacity=0.6,
                    ),
                    name=f'{index} - {country}',
                    showlegend=False,
                    visible=visible,
                    legendgroup=country
                ),
                row=row, col=col
            )
            
            # Add regression line if we have enough data points
            if len(country_df) > 2:
                # Calculate regression line
                x = country_df[index].values
                y = country_df['uncertainty'].values
                mask = ~np.isnan(x) & ~np.isnan(y)
                if sum(mask) > 1:  # Need at least 2 points for regression
                    x = x[mask]
                    y = y[mask]
                    slope, intercept = np.polyfit(x, y, 1)
                    x_range = np.linspace(min(x), max(x), 100)
                    y_range = slope * x_range + intercept
                    
                    # Add trend line
                    fig.add_trace(
                        go.Scatter(
                            x=x_range,
                            y=y_range,
                            mode='lines',
                            line=dict(color='red', width=2),
                            showlegend=False,
                            visible=visible,
                            legendgroup=country
                        ),
                        row=row, col=col
                    )
            
    
    # Create dropdown menu for country selection
    buttons = []
    
    for c_idx, country in enumerate(countries):
        # Calculate which traces belong to this country (each country has 2*len(indices) traces)
        # For each index: 1 scatter plot + 1 trend line
        traces_per_country = 2 * len(indices)
        visible_traces = [False] * (traces_per_country * len(countries))
        
        # Set the traces for the selected country to visible
        for i in range(traces_per_country):
            visible_traces[c_idx * traces_per_country + i] = True
        
        button = dict(
            label=country,
            method="update",
            args=[
                {"visible": visible_traces},
                {"title": f"Uncertainty vs Acoustic Indices - {country}"}
            ]
        )
        buttons.append(button)
    
    fig.update_layout(
        height=250*n_rows,
        width=1000,
        title="Uncertainty vs Acoustic Indices by Country",
        updatemenus=[dict(
            active=0,
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )],
        annotations=[
            dict(text="Select Country:",
                 x=0.01,
                 y=1.1,
                 xref="paper",
                 yref="paper",
                 showarrow=False)
        ]
    )
    
    # Update axes labels
    for i, index in enumerate(indices):
        row = i // n_cols + 1
        col = i % n_cols + 1
        fig.update_xaxes(title_text=index, row=row, col=col)
        fig.update_yaxes(title_text="Uncertainty" if col == 1 else "", row=row, col=col)
    
    return fig

# Create and display the plot
indices = ['ACI', 'ADI', 'BI', 'Hf', 'NDSI', 'EVNspCount']
fig = create_uncertainty_vs_indices_plotly(agg_df, indices)
fig.show()